# **Cluster Management**

This notebooke is to assist with managing the your cluster.

## Available Machines

This is from the canvas announcement.

### Freddy

**Ports:** [30785, 30799] 

1. columbia 
2. columbus-oh 
3. concord 
4. denver 
5. des-moines 
6. dover 
7. frankfort 
8. harrisburg 
9. hartford 
10. helena 
11. honolulu 
12. indianapolis 

### Eric

*idk*

In [1]:
# Specify your user name
username = 'flarrieu'

In [2]:
# Current Machines and port in use

with open(f'/s/chopin/a/grad/{username}/bigdata_configs/machines_and_ports.txt') as f: 
    s = f.read()
    print(s)

columbus-oh
concord
columbia
denver
des-moines
dover
frankfort
harrisburg
30785




In [3]:
# Machine Roles After Cluster Setup

with open(f'/s/chopin/a/grad/{username}/bigdata_configs/README.md', 'r') as file:
    lines = file.readlines()
    
machine_roles = lines[-8:]

for machine in machine_roles:
    print(machine)
    if "namenode" in machine and not "secondary" in machine:
        namenode = machine.split(" ")[2]
    if "resource manager" in machine:
        resource_manager = machine.split(" ")[3]

- namenode: columbus-oh

- secondary namenode: concord

- resource manager: columbia

- datanode1: denver

- datanode2: des-moines

- datanode3: dover

- datanode4: frankfort

- datanode5: harrisburg


# **Current Cluster Status**

In [4]:
%%bash
# Print the current status of the cluster
# You will need to set the following in your ~/.bashrc: 
#   chmod +x /s/chopin/a/grad/{username}/hadoopConf/monitor.sh
#   export MONITOR_CLUSTER="/s/chopin/a/grad/{username}/hadoopConf/monitor.sh"

source ~/.bashrc
$MONITOR_CLUSTER

/usr/share/Modules/init/bash: line 104: unalias: pip3: not found
/usr/share/Modules/init/bash: line 105: unalias: pip: not found


columbus-oh
1836893 Jps
142507 NameNode
1258639 Master
concord
1078098 SecondaryNameNode
1738807 Jps
columbia
761357 Jps
2347557 ResourceManager
denver
3270048 NodeManager
3897609 Worker
3897437 Worker
3269352 DataNode
1624645 Jps
des-moines
405071 Jps
175883 Worker
175742 Worker
dover
2698953 DataNode
3551319 Worker
3793325 Jps
3551171 Worker
2699470 NodeManager
frankfort
573610 DataNode
961685 Jps
861857 Worker
861729 Worker
574052 NodeManager
harrisburg
2338883 NodeManager
2338355 DataNode
3637996 Jps
3166939 Worker
3166811 Worker


## **Starting DFS and Yarn**

In [14]:
%pip install paramiko

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/python-env/py39/lib/python3.9/site-packages/p

In [6]:
import os
import paramiko

# You will need to set the following in your ~/.bashrc: 
#   export PASSWORD="your password"

password = os.getenv("PASSWORD")

if not password:
    print("Error: You need to set your password in you bashrc.")

In [28]:
# Start DFS
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/start-dfs.sh') # Run start-dfs.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

Starting namenodes on [columbus-oh.cs.colostate.edu]
Starting datanodes
Starting secondary namenodes [concord.cs.colostate.edu]




In [29]:
# Start Yarn
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(resource_manager.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/hadoop/latest/sbin/start-yarn.sh') # Run start-yarn.sh on resource manager
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

Starting resourcemanager
Starting nodemanagers




## **Stopping Master and Workers**

In [13]:
if password:
    sure = input("Are you sure: y or n?")
    if sure == "y":
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(namenode.strip(), username=username, password=password)
        stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/stop-master.sh') # Run stop-dfs.sh on namenode
        print(stdout.read().decode())
        print(stderr.read().decode())

ssh.close()    

stopping org.apache.spark.deploy.master.Master




In [12]:
if password:
    sure = input("Are you sure: y or n?")
    if sure == "y":
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(resource_manager.strip(), username=username, password=password)
        stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/stop-workers.sh') # Run start-yarn.sh on resource manager
        print(stdout.read().decode())
        print(stderr.read().decode())

ssh.close()    

des-moines: stopping org.apache.spark.deploy.worker.Worker
des-moines: stopping org.apache.spark.deploy.worker.Worker
frankfort: stopping org.apache.spark.deploy.worker.Worker
frankfort: stopping org.apache.spark.deploy.worker.Worker
denver: stopping org.apache.spark.deploy.worker.Worker
denver: stopping org.apache.spark.deploy.worker.Worker
harrisburg: stopping org.apache.spark.deploy.worker.Worker
harrisburg: stopping org.apache.spark.deploy.worker.Worker
dover: stopping org.apache.spark.deploy.worker.Worker
dover: stopping org.apache.spark.deploy.worker.Worker




# **Starting Master and Workers**

In [14]:
# Start master
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/start-master.sh') # Run start-master.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

starting org.apache.spark.deploy.master.Master, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.master.Master-1-columbus-oh.out




In [15]:
# Start workers
if password:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(namenode.strip(), username=username, password=password)
    stdin, stdout, stderr = ssh.exec_command('bash /usr/local/spark/latest/sbin/start-workers.sh') # Run start-master.sh on namenode
    print(stdout.read().decode())
    print(stderr.read().decode())

ssh.close()    

denver: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.worker.Worker-1-denver.out
denver: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.worker.Worker-2-denver.out
frankfort: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.worker.Worker-1-frankfort.out
frankfort: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.worker.Worker-2-frankfort.out
harrisburg: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/logs/spark-flarrieu-org.apache.spark.deploy.worker.Worker-1-harrisburg.out
harrisburg: starting org.apache.spark.deploy.worker.Worker, logging to /s/chopin/a/grad/flarrieu/sparkConf/log

# **Clients**

In [34]:
import xml.etree.ElementTree as ET

In [35]:
# Parsing .xml
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    properties = {}
    for property in root.findall('property'):
        name = property.find('name').text
        value = property.find('value').text
        properties[name] = value
    return properties

In [36]:
# Parsing .conf
def parse_spark_conf(file_path):
    properties = {}
    with open(file_path, 'r') as file:
        for line in file:
            if 'spark.master' in line and not "#" in line:
                key, value = line.split(" ", 1)
                properties[key.strip()] = value.strip()
                
    return properties

In [37]:
# Parsing .env
def parse_spark_env(file_path):
    properties = {}
    with open(file_path, 'r') as file:
        for line in file:
            if 'export SPARK_MASTER_WEBUI_PORT' in line:
                key, value = line.split("=", 1)
                properties['port'] = value.split(" ")[0]
                pass
    return properties

In [38]:
hdfs_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/hdfs-site.xml"
core_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/core-site.xml"
yarn_site_path = "/s/chopin/a/grad/flarrieu/hadoopConf/yarn-site.xml"
spark_conf_path = "/s/chopin/a/grad/flarrieu/sparkConf/spark-defaults.conf"
spark_env_path = "/s/chopin/a/grad/flarrieu/sparkConf/spark-env.sh"

hdfs_properties = parse_xml(hdfs_site_path)
core_properties = parse_xml(core_site_path)
yarn_properties = parse_xml(yarn_site_path)
spark_properties = parse_spark_conf(spark_conf_path)
spark_env_properties = parse_spark_env(spark_env_path)

print("HDFS Client")
print("http://" + hdfs_properties.get("dfs.namenode.http-address"))

print("Core Site")
print(core_properties.get('fs.default.name'))

print("Resource Management Client")
print("http://" + resource_manager.strip() + ".cs.colostate.edu:" + yarn_properties.get('yarn.resourcemanager.webapp.address').split(":")[1])

print("Spark")
print(spark_properties.get('spark.master'))

print("Spark Client")
print("http://" + namenode.strip() + ".cs.colostate.edu:" + spark_env_properties.get("port"))

HDFS Client
http://columbus-oh.cs.colostate.edu:30786
Core Site
hdfs://columbus-oh.cs.colostate.edu:30785
Resource Management Client
http://columbia.cs.colostate.edu:30798
Spark
spark://columbus-oh.cs.colostate.edu:30800
Spark Client
http://columbus-oh.cs.colostate.edu:30801
